# Computational Theory: Analysis of the Countdown Numbers Game

*Jakub Prochnicki G00373793*

---

# Table of Contents
1. [Introduction](#intro)
2. [Explanation of the Countdown Numbers Game](#concepts)
3. [Analysis of the Computational Complexity](#oracle)
4. [Approach to Solving the Game](#implementation)
5. [Conclusion](#conclusion)


---

# Introduction
---

This notebook constitutes my assessment submission for the Computational Theory Module at ATU Galway. The aim of this notebook is to provide an analysis of the Countdown Numbers Game. Firstly, I will provide an explanation of the game and give an example of the game in action. Following the explanation, I delve into an analysis of the computational complexity of the game and provide various approaches to solving the game, namely: Brute Force Approach, Heuristic Approach and Reverse Polish Notation. Lastly, I will evaluate my findings from solving the problem using various approaches